In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5, 0.5, 0.5))
])

In [3]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|██████████| 170M/170M [00:03<00:00, 42.7MB/s]


In [4]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size= 32, shuffle= True , num_workers = 2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size= 32, shuffle= True , num_workers = 2)

In [5]:
image, label = train_data[0]

In [6]:
image.size()

torch.Size([3, 32, 32])

In [7]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [8]:
class NeuralNet(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(3, 12, 5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(12, 24, 5)
    self.fc1 = nn.Linear(24*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x,1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x



In [9]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [10]:
for epoch in range(30):
  print (f'Training epoch {epoch+1}')

  running_loss = 0.0
  for i, data in enumerate(train_loader):
    inputs, labels = data
    optimizer.zero_grad()
    outputs = net(inputs)

    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print(f'loss: {running_loss/len(train_loader):.4f}')

Training epoch 1
loss: 2.2591
Training epoch 2
loss: 1.8078
Training epoch 3
loss: 1.5612
Training epoch 4
loss: 1.4226
Training epoch 5
loss: 1.3305
Training epoch 6
loss: 1.2476
Training epoch 7
loss: 1.1831
Training epoch 8
loss: 1.1273
Training epoch 9
loss: 1.0702
Training epoch 10
loss: 1.0222
Training epoch 11
loss: 0.9772
Training epoch 12
loss: 0.9357
Training epoch 13
loss: 0.8980
Training epoch 14
loss: 0.8606
Training epoch 15
loss: 0.8256
Training epoch 16
loss: 0.7952
Training epoch 17
loss: 0.7651
Training epoch 18
loss: 0.7346
Training epoch 19
loss: 0.7119
Training epoch 20
loss: 0.6845
Training epoch 21
loss: 0.6590
Training epoch 22
loss: 0.6337
Training epoch 23
loss: 0.6081
Training epoch 24
loss: 0.5860
Training epoch 25
loss: 0.5649
Training epoch 26
loss: 0.5416
Training epoch 27
loss: 0.5210
Training epoch 28
loss: 0.4961
Training epoch 29
loss: 0.4789
Training epoch 30
loss: 0.4548


In [12]:
torch.save(net.state_dict(), 'trained_net.pth')

In [13]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [14]:
correct = 0
total = 0

net.eval()
with torch.no_grad():
  for data in test_loader:
    images, labels = data
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()


accuracy  = 100 * correct / total
print(f'Accuracy of the network on the 10000 test images: {accuracy} %')

Accuracy of the network on the 10000 test images: 68.92 %


In [ ]:
new_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

def lpad_image(image_path):
  image = Image.open(image_path)
  image = new_transform(image)
  image = image.unsqueeze(0)
  return image


